In [54]:
import pandas as pd

data = pd.read_csv('AUCs.txt', sep='\t')

In [55]:
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [56]:
data.columns[1:]

Index(['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf'], dtype='object')

In [57]:
from scipy import stats
corr, p = stats.mannwhitneyu(data['C4.5'], data['C4.5+m'])
p

0.3228781552767225

In [58]:
import pandas as pd
import numpy as np
from scipy import stats

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

corr_data = []

for i, lhs_column in enumerate(data.columns[1:]):
    for j, rhs_column in enumerate(data.columns[1:]):
        if i >= j:
            continue

        corr, p = stats.wilcoxon(data[lhs_column], data[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

In [59]:
data_correlation = pd.DataFrame.from_records(corr_data)
data_correlation.columns = ['A', 'B', 'corr', 'p']

In [60]:
data_correlation

,A,B,corr,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


In [61]:
amount = 0
for i in range(len(corr_data)):
    if corr_data[i][3] < 0.05:
        print(corr_data[i][3])
        amount += 1
print(amount)

0.01075713311978963
0.015906444101703374
0.046332729793395394
0.022909099354356588
4


In [62]:
reject, p_corrected, a1, a2 = multipletests(data_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [63]:
data_correlation['p_corrected'] = p_corrected
data_correlation['reject'] = reject

In [64]:
data_correlation

,A,B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [65]:
reject, p_corrected, a1, a2 = multipletests(data_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [66]:
data_correlation['p_corrected'] = p_corrected
data_correlation['reject'] = reject

In [67]:
data_correlation

,A,B,corr,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True
